In [1]:
words = open("names.txt","r").read().splitlines()
print(words[:5],len(words))

['emma', 'olivia', 'ava', 'isabella', 'sophia'] 32033


In [2]:
for w in words:
    if len(w) <=2:
    
        words.remove(w)
len(words)

31970

In [3]:
sorted_alphabets = sorted(list(set("".join(words))))
stoi = {}
for i , j in enumerate(sorted_alphabets):
    stoi[j] = i+1
stoi["."] = 0
print(stoi)
itos = {}
for i , j in enumerate(sorted_alphabets):
    itos[i+1] = j
itos[0] = "."
print(itos)





{'a': 1, 'b': 2, 'c': 3, 'd': 4, 'e': 5, 'f': 6, 'g': 7, 'h': 8, 'i': 9, 'j': 10, 'k': 11, 'l': 12, 'm': 13, 'n': 14, 'o': 15, 'p': 16, 'q': 17, 'r': 18, 's': 19, 't': 20, 'u': 21, 'v': 22, 'w': 23, 'x': 24, 'y': 25, 'z': 26, '.': 0}
{1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z', 0: '.'}


In [4]:
import torch

In [5]:
N = torch.ones(size = (27,27,27))


In [6]:

for w in words:
    
    chs = ["."] + ["."] + list(w) + ["."]
    
    for ch1,ch2,ch3 in zip(chs,chs[1:],chs[2:]):
        ix1 = stoi[ch1]
        ix2 = stoi[ch2]
        ix3 = stoi[ch3]
        N[ix1,ix2,ix3] +=1
        

In [7]:
N[0,0,0]

tensor(1.)

In [8]:
# we need to normalize the counts
# sum when the dimension is 1
sum1 = N.sum(dim=2,keepdim=True)
sum1.shape


torch.Size([27, 27, 1])

In [9]:
N /= sum1
N.shape

torch.Size([27, 27, 27])

In [10]:
g= torch.Generator().manual_seed(2147483647)
from collections import Counter


for i in range(10):
    numbers = []
    ix1=0
    ix2=0
    while True:
        
        ix = torch.multinomial(N[ix1,ix2,:],num_samples=1,replacement=True,generator=g).item()
        if itos[ix]==".":
            break
        ix1=ix2
        ix2=ix
        numbers.append(itos[ix])
        
    print("".join(numbers))

ce
bra
jalius
rochityharlonimittain
luwak
ka
da
samiyah
javer
gotai


### Loss of this model

In [11]:
log_prob=0.0
n=0
for w in words:
    chs = ["."] + ["."] + list(w) + ["."]
    for ch1,ch2,ch3 in zip(chs,chs[1:],chs[2:]):
        ix1 = stoi[ch1]
        ix2 = stoi[ch2]
        ix3 = stoi[ch3]
        log_prob += torch.log(N[ix1,ix2,ix3])
        n+=1
print("Negative log likelihood of this network is : ", -log_prob.item()/n)     
        

Negative log likelihood of this network is :  2.2112069107331647


## Now we will build the neural network 

In [39]:
xs,ys = [],[]
for w in words:
    chs = ["."] + ["."] + list(w) + ["."]
    for ch1,ch2,ch3 in zip(chs,chs[1:],chs[2:]):
        xs.append([stoi[ch1],stoi[ch2]])
        ys.append(stoi[ch3])
xs = torch.tensor(xs)
ys = torch.tensor(ys)
ys.shape

torch.Size([227957])

In [46]:
import torch.nn.functional as F
xenc = F.one_hot(xs, num_classes=27).float().reshape((227957,54))
print(xenc[0],xenc.shape)


tensor([1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]) torch.Size([227957, 54])


In [81]:
g= torch.Generator().manual_seed(2147483647)
W = torch.randn(size=(54,27),requires_grad=True,generator=g)


In [ ]:

ix=0
name = []
g= torch.Generator().manual_seed(2147483647)

logits = xenc @ W
counts = logits.exp()
prob = counts / counts.sum(dim=1,keepdim=True)

prob.shape
prob[0]

tensor([0.0003, 0.1018, 0.0324, 0.0190, 0.0417, 0.0288, 0.0618, 0.2490, 0.0074,
        0.1235, 0.0169, 0.0012, 0.0341, 0.0043, 0.0297, 0.0164, 0.0171, 0.0280,
        0.0376, 0.0566, 0.0048, 0.0008, 0.0157, 0.0068, 0.0211, 0.0187, 0.0247],
       grad_fn=<SelectBackward0>)

In [79]:
loss = -prob[torch.arange(227957), ys].log().mean() + 0.1 * (W**2).mean()
loss

tensor(4.2928, grad_fn=<AddBackward0>)

In [80]:
W.grad = None
loss.backward()

W.data -=  W.grad

### Building the full GD

In [115]:
for i in range(1000):
    # forward pass
    logits = xenc @ W
    # Calculate softmax
    counts = logits.exp()
    prob = counts / counts.sum(dim=1,keepdim=True)

    # calculate loss
    loss = -prob[torch.arange(227957), ys].log().mean() 
    if i%10==0:
        print(i,loss)
    
    # backpropogation
    W.grad = None
    loss.backward()
    W.data -=  10* W.grad


0 tensor(2.3371, grad_fn=<NegBackward0>)
10 tensor(2.3371, grad_fn=<NegBackward0>)
20 tensor(2.3371, grad_fn=<NegBackward0>)
30 tensor(2.3371, grad_fn=<NegBackward0>)
40 tensor(2.3371, grad_fn=<NegBackward0>)
50 tensor(2.3371, grad_fn=<NegBackward0>)
60 tensor(2.3371, grad_fn=<NegBackward0>)
70 tensor(2.3371, grad_fn=<NegBackward0>)
80 tensor(2.3371, grad_fn=<NegBackward0>)
90 tensor(2.3371, grad_fn=<NegBackward0>)
100 tensor(2.3371, grad_fn=<NegBackward0>)
110 tensor(2.3371, grad_fn=<NegBackward0>)
120 tensor(2.3371, grad_fn=<NegBackward0>)
130 tensor(2.3370, grad_fn=<NegBackward0>)
140 tensor(2.3370, grad_fn=<NegBackward0>)
150 tensor(2.3370, grad_fn=<NegBackward0>)
160 tensor(2.3370, grad_fn=<NegBackward0>)
170 tensor(2.3370, grad_fn=<NegBackward0>)
180 tensor(2.3370, grad_fn=<NegBackward0>)
190 tensor(2.3370, grad_fn=<NegBackward0>)
200 tensor(2.3370, grad_fn=<NegBackward0>)
210 tensor(2.3370, grad_fn=<NegBackward0>)
220 tensor(2.3370, grad_fn=<NegBackward0>)
230 tensor(2.3370, gra

In [116]:
w = "bhavya"
test = []
validate = []
chs = ["."] + ["."] + list(w) + ["."]
for ch1,ch2,ch3 in zip(chs,chs[1:],chs[2:]):
    ix1 = stoi[ch1]
    ix2 = stoi[ch2]
    ix3 = stoi[ch3]
    test.append([ix1,ix2])
    validate.append(ix3)

test = torch.tensor(test)
validate = torch.tensor(validate)
print(test.shape)
print(validate.shape)
n = test.shape[0]


torch.Size([7, 2])
torch.Size([7])


In [117]:
test_x = F.one_hot(test,num_classes=27).reshape((n,54)).float()
test_x.shape

torch.Size([7, 54])

In [118]:
# forward pass
logits = test_x @ W     # multiply by weights

# calculate softmax
count = logits.exp()
prob = count/count.sum(dim=1,keepdim=True)

loss = -prob[torch.arange(n),validate].log().mean()


In [119]:
print(loss)

tensor(2.5378, grad_fn=<NegBackward0>)


## Final Validation loss is 2.4636 of trained model

In [120]:
g = torch.Generator().manual_seed(2147483647)
for i in range(10):
    ix1 = 0
    ix2 = 0
    word = []
    while True:
        input_data = F.one_hot(torch.tensor([ix1,ix2]),num_classes=27).float().reshape(1,54)
        logits = input_data @ W
        count = logits.exp()
        prob = count/count.sum(dim=1,keepdim=True)
        ix = torch.multinomial(prob,num_samples=1,replacement=True,generator=g).item()
        word.append(itos[ix])
        if ix ==0:
            break
        ix1 = ix2
        ix2 = ix

    print("".join(word))




ce.
braidallurailazitynnellin.
jana.
nallayn.
ka.
da.
samiyaubrtthrigotai.
morielliaugie.
teda.
kaleyla.
